<h1 style='color:green;'><center>Long short-term memory (LSTM)</center></h1>

<em>In this notebook, we train a Long Short-Term Memory (LSTM) model on the BBC news classification problem. The goal is to classify news articles into predefined categories based on their content. We preprocess the text data, build and train the LSTM model, and evaluate its performance on a test dataset.</em>

In [2]:
import pandas as pd 
import numpy as np 

df = pd.read_csv("../Assets/bbc-news-data.csv")
df.head()

,category,title,content
0,business,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...


In [41]:

df['category'].value_counts()


category
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: count, dtype: int64

In [1]:

import keras as kr
from keras.src.layers import SimpleRNN, Dense, Embedding, LSTM
from keras.src.models import Sequential
from keras.api.preprocessing.sequence import pad_sequences
from keras._tf_keras.keras.preprocessing.text import Tokenizer as tok
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re


In [3]:
## methos to clean the text data in dataframe

def text_preprocess(text):
    
    if text:
        text_to_lower = text.lower()
        clean_text = re.sub("[^a-zA-Z]", " ", text_to_lower)
        test_to_list = clean_text.split()
        stem= WordNetLemmatizer()
        tokens = [ stem.lemmatize(word=word) for word in test_to_list if not word in set(stopwords.words('english'))]
        tokens_to_str = " ".join(tokens)
        return tokens_to_str    
    
        

In [4]:
df['title'] = df['title'].apply(lambda x : text_preprocess(x))

In [5]:
df.head()

,category,title,content
0,business,ad sale boost time warner profit,Quarterly profits at US media giant TimeWarne...
1,business,dollar gain greenspan speech,The dollar has hit its highest level against ...
2,business,yukos unit buyer face loan claim,The owners of embattled Russian oil giant Yuk...
3,business,high fuel price hit ba profit,British Airways has blamed high fuel prices f...
4,business,pernod takeover talk lift domecq,Shares in UK drinks and food firm Allied Dome...


In [6]:
## checking for max length of sentence
max(df['title'].str.split(' ').str.len())

7

In [7]:
X = df['title']
y = df['category']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [8]:
x_train.shape, x_test.shape

((1557,), (668,))

In [9]:
y_train.shape, y_test.shape

((1557,), (668,))

In [11]:
y_train_dummy = pd.get_dummies(y_train)
y_test_dummy = pd.get_dummies(y_test)

y_train.shape

(1557,)

In [26]:
def tokenize_and_padding(input_text, max_lenght, tok):
    text_seq = tok.texts_to_sequences(input_text)
    tokens = pad_sequences(text_seq, max_lenght, padding="post")
    return (tokens)


text_tok = tok()
text_tok.fit_on_texts(x_train)

x_train_tokens = tokenize_and_padding(input_text=x_train, max_lenght=7, tok=text_tok)
x_test_tokens = tokenize_and_padding(input_text=x_test,max_lenght=7, tok=text_tok)

In [27]:
x_train_tokens


array([[1135,  191,  124, ...,    0,    0,    0],
       [ 452,  453,  656, ...,    0,    0,    0],
       [1138,  453, 1139, ...,    0,    0,    0],
       ...,
       [ 148,  180,    2, ..., 2683,  104,    0],
       [  15,   77,  389, ...,   27,    0,    0],
       [ 155,    8, 2685, ...,    0,    0,    0]])

In [29]:
x_test_tokens

array([[   4,  317,   90, ...,    0,    0,    0],
       [ 248,   12,  290, ...,   17,    0,    0],
       [ 560,  231,  685, ...,    0,    0,    0],
       ...,
       [1577, 1578,  906, ...,   67,    0,    0],
       [ 892,  302,  504, ...,    0,    0,    0],
       [  22,  336,  160, ...,    0,    0,    0]])

In [30]:
voc_size=len(text_tok.word_index)+1
Embedding_dim = 70

In [37]:
## creating model
from keras.src.regularizers import L2

model = Sequential()
model.add(Embedding(voc_size, Embedding_dim, trainable=True))
model.add(SimpleRNN(units=40, activation='tanh' , activity_regularizer=L2(0.01)))
model.add(Dense(5, activation='softmax'))

In [38]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [39]:
y_train_dummy.shape

(1557, 5)

In [40]:
model.fit(x_train_tokens, y_train_dummy,batch_size=64, epochs=50, validation_data=(x_test_tokens, y_test_dummy))

Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 18s 146ms/step - accuracy: 0.2757 - loss: 1.5836 - val_accuracy: 0.3428 - val_loss: 1.5064
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.7113 - loss: 1.1784 - val_accuracy: 0.6437 - val_loss: 1.0300
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - accuracy: 0.9471 - loss: 0.4023 - val_accuracy: 0.7275 - val_loss: 0.7921
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9964 - loss: 0.0949 - val_accuracy: 0.7365 - val_loss: 0.8036
Epoch 5/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - accuracy: 0.9969 - loss: 0.0407 - val_accuracy: 0.7440 - val_loss: 0.8463
Epoch 6/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 1.0000 - loss: 0.0193 - val_accuracy: 0.7410 - val_loss: 0.8721
Epoch 7/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 1.0000 - loss: 0.0130 - val_accuracy: 0.7320 - val_loss: 0.9097
Epoch 8/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 1.0000 - loss: 0.0098 - val_accuracy: 0.7275 -